In [9]:
import fitz
import pandas as pd
from datetime import datetime
import logging


logging.basicConfig(level=logging.DEBUG)
# filename = 'data/fwil_dhi_me_results_semi.pdf'
# filename = 'data/fwil_dhi_me_results_qr.pdf'
# filename = 'data/fwil_dhi_me_results_f.pdf'
# filename = 'data/fwil_dhi_me_results_tt.pdf'
# filename = 'data/leog_dhi_me_results_qr.pdf'
# filename = 'data/leog_dhi_me_results_semi.pdf'
# filename = 'data/leog_dhi_me_results_f.pdf'
filename = 'data/vdso_dhi_me_results_semi.pdf'
# filename = 'data/vdso_dhi_me_results_f.pdf'
# filename = 'data/vdso_dhi_me_results_qr.pdf'



In [10]:
doc = fitz.open(filename)
page = doc[0]
text = page.get_text("text")

# Identify where the table starts 25 for the semis and 24 for the qualifiers 24 for the finals and 34 for the time trials
text.split('\n')[24:64]


['Time Points',
 '1.',
 '20 PIERRON Amaury',
 'PIERRON Amaury',
 'PIERRON Amaury',
 'PIERRON Amaury',
 'COMMENCAL/MUC-OFF BY RIDING',
 '10008827283',
 'FRA',
 '1996',
 '59.875 (1)',
 '0:46.294 (1)',
 '1:45.544 (2)',
 '2:34.042 (2)',
 '3:08.938 (1)',
 '3:44.312',
 '3:44.312',
 '3:44.312',
 '3:44.312',
 '+0.000',
 '100',
 '2. P',
 '4 ILES Finn',
 'ILES Finn',
 'ILES Finn',
 'ILES Finn',
 'SPECIALIZED GRAVITY',
 '10090907774',
 'CAN',
 '1999',
 '56.118 (25)',
 '0:46.342 (2)',
 '1:45.357 (1)',
 '2:33.099 (1)',
 '3:10.160 (2)',
 '3:45.313',
 '3:45.313',
 '3:45.313',
 '3:45.313',
 '+1.001']

In [11]:
import fitz
from typing import List, Dict, Union
import pandas as pd
from datetime import datetime, timedelta

filename = filename
table_start_line = 25

def extract_time_and_rank(data_string: str) -> (str, str):
    if "(" in data_string:
        time, rank = data_string.split()[0], data_string.split()[-1].strip("()")
        return time, rank
    else:
        return "N/A", "N/A"

def calculate_sector_times(split_times: List[str]) -> List[str]:
    sector_times = []
    previous_time = "0:00.000"

    for split_time in split_times:
        try:
            delta = datetime.strptime(split_time, "%M:%S.%f") - datetime.strptime(previous_time, "%M:%S.%f")
            sector_times.append(str(delta)[2:])  # Skip "0:" part in "0:XX.XXX" string
            previous_time = split_time
        except ValueError:
            sector_times.append("N/A")

    return sector_times

def extract_rider_info_all_pages(filename: str, table_start_line: int = 25) -> List[Dict[str, Union[str, List[str]]]]:
    doc = fitz.open(filename)
    riders_info = []
    for page_num in range(len(doc)):
        page = doc[page_num]
        text = page.get_text("text")
        lines = text.split('\n')
        line_start = table_start_line

        while line_start < len(lines):
            rider_info = lines[line_start:line_start + 20]
            if len(rider_info) < 19:
                break

            # No team case
            if rider_info[5].isdigit():
                speed_trap, speed_trap_rank = extract_time_and_rank(rider_info[8])
                split_times, split_time_ranks = zip(*(extract_time_and_rank(s) for s in rider_info[9:13]))
                rider_data = {
                    'rank': rider_info[0].split()[0].replace('.', ''),
                    'protected': rider_info[0].split()[1] if len(rider_info[0].split()) > 1 else '',
                    'rider_number': rider_info[1].split()[0],
                    'name': ' '.join(rider_info[1].split()[1:]),
                    'team': 'N/A',
                    'uci_id': rider_info[5],
                    'country': rider_info[6],
                    'birth_year': rider_info[7],
                    'speed_trap': speed_trap,
                    'speed_trap_rank': speed_trap_rank,
                    'split_times': list(split_times),
                    'split_time_ranks': list(split_time_ranks),
                    'final_time': rider_info[13],
                    'gap': rider_info[17] if len(rider_info) > 17 else 'N/A',
                    'points': rider_info[18] if len(rider_info) > 18 else 'N/A'
                }
                next_offset = 19
            # With team case
            else:
                speed_trap, speed_trap_rank = extract_time_and_rank(rider_info[9])
                split_times, split_time_ranks = zip(*(extract_time_and_rank(s) for s in rider_info[10:14]))
                rider_data = {
                    'rank': rider_info[0].split()[0].replace('.', ''),
                    'protected': rider_info[0].split()[1] if len(rider_info[0].split()) > 1 else '',
                    'rider_number': rider_info[1].split()[0],
                    'name': ' '.join(rider_info[1].split()[1:]),
                    'team': rider_info[5],
                    'uci_id': rider_info[6],
                    'country': rider_info[7],
                    'birth_year': rider_info[8],
                    'speed_trap': speed_trap,
                    'speed_trap_rank': speed_trap_rank,
                    'split_times': list(split_times),
                    'split_time_ranks': list(split_time_ranks),
                    'final_time': rider_info[14],
                    'gap': rider_info[18] if len(rider_info) > 18 else 'N/A',
                    'points': rider_info[19] if len(rider_info) > 19 else 'N/A'
                }
                next_offset = 20

            if rider_data['final_time'] in ['DNF', 'DNS']:
                break

            sector_times = calculate_sector_times(rider_data['split_times'])
            rider_data['sector_times'] = sector_times
            riders_info.append(rider_data)
            line_start += next_offset

    return riders_info

# Generate DataFrame
riders_info = extract_rider_info_all_pages(filename, table_start_line)

df = pd.DataFrame(riders_info)
for i in range(4):
    df[f'split_{i+1}'] = df['split_times'].apply(lambda x: x[i] if len(x) > i else 'N/A')
    df[f'split_{i+1}_rank'] = df['split_time_ranks'].apply(lambda x: x[i] if len(x) > i else 'N/A')
    df[f'sector_{i+1}'] = df['sector_times'].apply(lambda x: x[i] if len(x) > i else 'N/A')

# Rank the sector times correctly
for i in range(4):
    df[f'sector_{i+1}_rank'] = df[f'sector_{i+1}'].apply(
        lambda x: timedelta(minutes=int(x.split(":")[0]), seconds=float(x.split(":")[1]))
        if x not in ["N/A", "-"]
        else timedelta.max
    ).rank(method="min").astype(int)

# Handle the final sector (sector_5)
df[f'sector_5'] = df.apply(lambda row: 
    str(datetime.strptime(row["final_time"], "%M:%S.%f") - datetime.strptime(row["split_4"], "%M:%S.%f"))[2:]
    if row["final_time"] not in ["DNF", "DNS", "N/A", "-"] and row["split_4"] not in ["DNF", "DNS", "N/A", "-"]
    else "N/A", axis=1)

df[f'sector_5_rank'] = df[f'sector_5'].apply(
    lambda x: timedelta(minutes=int(x.split(":")[0]), seconds=float(x.split(":")[1]))
    if x not in ["N/A", "-"]
    else timedelta.max
).rank(method="min").astype(int)

df.drop(columns=['split_times', 'split_time_ranks', 'sector_times'], inplace=True)

# Display and Save
display(df)
file_prefix = filename.split('/')[-1].split('.')[0]
df.to_csv(f'data/{file_prefix}.csv', index=False)


,rank,protected,rider_number,name,team,uci_id,country,birth_year,speed_trap,speed_trap_rank,...,sector_3,split_4,split_4_rank,sector_4,sector_1_rank,sector_2_rank,sector_3_rank,sector_4_rank,sector_5,sector_5_rank
0,1,,20,PIERRON Amaury,COMMENCAL/MUC-OFF BY RIDING,10008827283,FRA,1996,59.875,1,...,00:48.498000,3:08.938,1,00:34.896000,1,2,2,2,00:35.374000,9
1,2,P,4,ILES Finn,SPECIALIZED GRAVITY,10090907774,CAN,1999,56.118,25,...,00:47.742000,3:10.160,2,00:37.061000,2,1,1,37,00:35.153000,3
2,3,P,8,BROSNAN Troy,CANYON CLLCTV FACTORY TEAM,10007307417,AUS,1993,57.669,7,...,00:49.579000,3:10.827,3,00:34.594000,4,3,5,1,00:34.832000,1
3,4,P,3,VERGIER Loris,TREK FACTORY RACING GRAVITY,10008723112,FRA,1996,59.333,2,...,00:50.887000,3:14.633,4,00:35.883000,7,6,19,9,00:35.286000,5
4,5,P,5,KOLB Andreas,CONTINENTAL ATHERTON,10009187092,AUT,1996,57.715,6,...,00:50.937000,3:15.191,6,00:35.485000,13,9,21,3,00:35.665000,13
5,6,,21,O CALLAGHAN Oisin *,YT MOB,10017486353,IRL,2003,56.228,22,...,00:50.972000,3:15.977,8,00:35.723000,16,13,22,7,00:35.293000,6
6,7,,27,THIRION Rémi,GIANT FACTORY OFF-ROAD TEAM - DH,10005415715,FRA,1990,55.684,31,...,00:50.657000,3:15.971,7,00:36.684000,17,7,14,27,00:35.955000,16
7,8,P,10,SHAW Luca,CANYON CLLCTV FACTORY TEAM,10008813442,USA,1996,58.920,3,...,00:51.034000,3:16.895,14,00:35.519000,26,14,23,5,00:35.239000,4
8,9,,48,BREEDEN Joe,INTENSE FACTORY RACING,10011005743,GBR,1999,55.879,29,...,00:50.006000,3:14.956,5,00:36.250000,12,8,8,15,00:37.304000,45
9,10,,51,CHAPELET Simon *,CUBE FACTORY RACING,10071553749,FRA,2002,56.009,27,...,00:51.395000,3:16.328,11,00:36.001000,11,10,26,11,00:36.151000,20


In [14]:
import fitz
from typing import List, Dict, Union
import pandas as pd
from datetime import datetime, timedelta

filename = 'data/leog_dhi_me_results_qr.pdf'
table_start_line = 24

def extract_time_and_rank(data_string: str) -> (str, str):
    if "(" in data_string:
        time, rank = data_string.split()[0], data_string.split()[-1].strip("()")
        return time, rank
    else:
        return "N/A", "N/A"

def calculate_sector_times(split_times: List[str]) -> List[str]:
    sector_times = []
    previous_time = "0:00.000"

    for split_time in split_times:
        try:
            delta = datetime.strptime(split_time, "%M:%S.%f") - datetime.strptime(previous_time, "%M:%S.%f")
            sector_times.append(str(delta)[2:])  # Skip "0:" part in "0:XX.XXX" string
            previous_time = split_time
        except ValueError as e:
            sector_times.append("N/A")

    return sector_times

def extract_rider_info_all_pages(filename: str, table_start_line: int = 25) -> List[Dict[str, Union[str, List[str]]]]:
    doc = fitz.open(filename)
    riders_info = []
    for page_num in range(len(doc)):
        page = doc[page_num]
        text = page.get_text("text")
        lines = text.split('\n')
        line_start = table_start_line

        while line_start < len(lines):
            rider_info = lines[line_start:line_start + 20]
            if len(rider_info) < 19:
                break

            # No team case
            if rider_info[5].isdigit():
                speed_trap, speed_trap_rank = extract_time_and_rank(rider_info[8])
                split_times, split_time_ranks = zip(*(extract_time_and_rank(s) for s in rider_info[9:13]))
                rider_data = {
                    'rank': rider_info[0].split()[0].replace('.', ''),
                    'protected': rider_info[0].split()[1] if len(rider_info[0].split()) > 1 else '',
                    'rider_number': rider_info[1].split()[0],
                    'name': ' '.join(rider_info[1].split()[1:]),
                    'team': 'N/A',
                    'uci_id': rider_info[5],
                    'country': rider_info[6],
                    'birth_year': rider_info[7],
                    'speed_trap': speed_trap,
                    'speed_trap_rank': speed_trap_rank,
                    'split_times': list(split_times),
                    'split_time_ranks': list(split_time_ranks),
                    'final_time': rider_info[13],
                    'gap': rider_info[17] if len(rider_info) > 17 else 'N/A',
                    'points': rider_info[18] if len(rider_info) > 18 else 'N/A'
                }
                next_offset = 19
            # With team case
            else:
                speed_trap, speed_trap_rank = extract_time_and_rank(rider_info[9])
                split_times, split_time_ranks = zip(*(extract_time_and_rank(s) for s in rider_info[10:14]))
                rider_data = {
                    'rank': rider_info[0].split()[0].replace('.', ''),
                    'protected': rider_info[0].split()[1] if len(rider_info[0].split()) > 1 else '',
                    'rider_number': rider_info[1].split()[0],
                    'name': ' '.join(rider_info[1].split()[1:]),
                    'team': rider_info[5],
                    'uci_id': rider_info[6],
                    'country': rider_info[7],
                    'birth_year': rider_info[8],
                    'speed_trap': speed_trap,
                    'speed_trap_rank': speed_trap_rank,
                    'split_times': list(split_times),
                    'split_time_ranks': list(split_time_ranks),
                    'final_time': rider_info[14],
                    'gap': rider_info[18] if len(rider_info) > 18 else 'N/A',
                    'points': rider_info[19] if len(rider_info) > 19 else 'N/A'
                }
                next_offset = 20

            if rider_data['final_time'] in ['DNF', 'DNS', 'DSQ']:
                line_start += next_offset
                continue

            sector_times = calculate_sector_times(rider_data['split_times'])
            rider_data['sector_times'] = sector_times
            riders_info.append(rider_data)
            line_start += next_offset

    return riders_info

def is_valid_time_format(time_str):
    try:
        datetime.strptime(time_str, "%M:%S.%f")
        return True
    except ValueError:
        return False

def is_invalid_entry(entry):
    invalid_terms = ['DNF', 'DNS', 'DSQ', '-', 'N/A', 'Average', 'YOB', 'In', 'Year', 'MACDERMID', 'GUIONNET', 'TRUMMER', 'RIESCO', 'SCHLEBES', 'DORVAL AM COMMENCAL']
    return any(term in entry for term in invalid_terms)

def validate_and_clean_data(df):
    valid_rows = []
    for _, row in df.iterrows():
        if (is_valid_time_format(row["final_time"]) and is_valid_time_format(row["split_4"]) and
                not is_invalid_entry(row["final_time"]) and not is_invalid_entry(row["split_4"])):
            valid_rows.append(row)
    return pd.DataFrame(valid_rows)

# Generate DataFrame
riders_info = extract_rider_info_all_pages(filename, table_start_line)

df = pd.DataFrame(riders_info)
for i in range(4):
    df[f'split_{i+1}'] = df['split_times'].apply(lambda x: x[i] if len(x) > i else 'N/A')
    df[f'split_{i+1}_rank'] = df['split_time_ranks'].apply(lambda x: x[i] if len(x) > i else 'N/A')
    df[f'sector_{i+1}'] = df['sector_times'].apply(lambda x: x[i] if len(x) > i else 'N/A')

# Debugging: Print problematic sector times
print("Sector times with issues:")
print(df[['sector_1', 'sector_2', 'sector_3', 'sector_4', 'final_time', 'split_4']].head(20))

# Additional Debugging for final sector times
for i, row in df.iterrows():
    final_time = row["final_time"]
    split_4 = row["split_4"]
    if not is_valid_time_format(final_time) or is_invalid_entry(final_time):
        print(f"Invalid final time format at index {i}: {final_time}")
    if not is_valid_time_format(split_4) or is_invalid_entry(split_4):
        print(f"Invalid split_4 time format at index {i}: {split_4}")

# Validate and clean the data
df = validate_and_clean_data(df)

# Drop rows with invalid final_time or split_4 before ranking
df = df[df.apply(lambda row: is_valid_time_format(row["final_time"]) and is_valid_time_format(row["split_4"]), axis=1)]

# Further remove rows with any invalid sector times
for i in range(4):
    df = df[df.apply(lambda row: is_valid_time_format(row[f'sector_{i+1}']), axis=1)]

# Rank the sector times correctly
for i in range(4):
    df[f'sector_{i+1}_rank'] = df[f'sector_{i+1}'].apply(
        lambda x: timedelta(minutes=int(x.split(":")[0]), seconds=float(x.split(":")[1]))
        if is_valid_time_format(x) else timedelta.max
    ).rank(method="min").astype(int)

# Handle the final sector (sector_5)
def calculate_final_sector(row):
    try:
        final_time = datetime.strptime(row["final_time"], "%M:%S.%f")
        split_4 = datetime.strptime(row["split_4"], "%M:%S.%f")
        return str(final_time - split_4)[2:]
    except Exception as e:
        return "N/A"

df[f'sector_5'] = df.apply(calculate_final_sector, axis=1)

df[f'sector_5_rank'] = df[f'sector_5'].apply(
    lambda x: timedelta(minutes=int(x.split(":")[0]), seconds=float(x.split(":")[1]))
    if is_valid_time_format(x) else timedelta.max
).rank(method="min").astype(int)

# Ensure there are no infinity values in the ranking columns
for i in range(1, 6):
    sector_col = f'sector_{i}_rank'
    if sector_col in df.columns:
        df[sector_col] = df[sector_col].replace([float('inf'), -float('inf')], 0)

# Drop columns only if they exist
columns_to_drop = ['split_times', 'split_time_ranks', 'sector_times']
df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

file_prefix = filename.split('/')[-1].split('.')[0]
csv_path = f'data/{file_prefix}.csv'
df.to_csv(csv_path, index=False)

csv_path


Sector times with issues:
        sector_1      sector_2      sector_3      sector_4 final_time  \
0   00:33.636000  00:44.378000  00:39.516000  00:38.700000   3:09.336   
1   00:33.248000  00:44.414000  00:37.384000  00:40.567000   3:09.396   
2   00:32.896000  00:44.641000  00:39.406000  00:39.980000   3:10.337   
3   00:33.412000  00:44.863000  00:38.922000  00:40.618000   3:10.819   
4   00:33.315000  00:44.364000  00:39.634000  00:40.398000   3:11.185   
5   00:32.779000  00:44.751000  00:38.502000  00:41.347000   3:11.208   
6   00:33.563000  00:44.457000  00:38.855000  00:40.560000   3:11.785   
7   00:33.606000  00:44.238000  00:39.232000  00:41.047000   3:12.185   
8   00:33.545000  00:44.809000  00:39.341000  00:40.505000   3:12.381   
9   00:33.488000  00:45.676000  00:39.666000  00:40.709000   3:12.763   
10  00:33.788000  00:45.031000  00:38.382000  00:41.757000   3:12.906   
11  00:34.769000  00:44.847000  00:40.050000  00:39.591000   3:13.034   
12  00:33.403000  00:45.8

'data/leog_dhi_me_results_qr.csv'